# 0. Installation (one time job)

In [1]:
# !pip install git+git://github.com/gunthercox/ChatterBot.git@master
# !pip install -U spacy==2.1.3
# !python -m spacy link en_core_web_sm en
# !pip3 install pyyaml

  Cloning git://github.com/gunthercox/ChatterBot.git (to revision master) to /tmp/pip-req-build-eaz2xnk7
  Running command git clone -q git://github.com/gunthercox/ChatterBot.git /tmp/pip-req-build-eaz2xnk7
     |████████████████████████████████| 1.3 MB 2.2 MB/s eta 0:00:01
  Created wheel for ChatterBot: filename=ChatterBot-1.1.0a7-py2.py3-none-any.whl size=63715 sha256=c1d68cba4fcd736473174160ce99fc8984d4dacdd523e1f76a6a1e6bd86d934b
  Stored in directory: /tmp/pip-ephem-wheel-cache-pzs1_soy/wheels/cb/73/ce/3fdd8d4bf061ff926911abf09b9ce689c84849ff6427c0841c
Successfully built ChatterBot


# 1. Import Library

In [9]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer
import pandas as pd
import spacy

# 2. Basic Usage

# 2.1. Create a chatter bot called chatbot

In [10]:
nlp = spacy.load("en")

In [3]:
chatbot = ChatBot('chatbot')

# 2.2. Build chatterbot trainer

In [9]:
# Since chatbot has no knowledge now, we create a new trainer for chatbot
# There are several ways to train the chatbot. You can explore them in 'chatterbot.trainers'
# ChatterBotCorpusTrainer is one of the trainer classes
trainer = ChatterBotCorpusTrainer(chatbot)

In [14]:
# Train the chatbot based on the english corpus.
# The corpus is given by the Chatterbot package.
# You can also create your own corpus.
# Once you train the chatbot with this corpus, annotate the below line with '#' preventing from duplicate training
trainer.train("chatterbot.corpus.english.greetings")

Training greetings.yml: [####################] 100%


# 2.3. Simple example

In [15]:
# Get a response to an input statement
# If you try to present same questions with several times, you may get different answers. How can it get that?
response = chatbot.get_response("Hi")
print(response)

Hello


# 3. Create New Conversation for Chatterbot

In [16]:
# Let's see what happens when you ask the chatbot -- Is Sam a good guy?
response = chatbot.get_response("Is Sam a good guy?")
print(response)

What's up?


In [17]:
# It seemly gives a irrelevant answer for the question. Could you figure out why?
# That's because in the greeting corpus, no any conversation about if Sam is a good guy.
# So we need to train the chatbot again!
# This time we use the ListTrainer to train one conversation

trainer = ListTrainer(chatbot)
train_list = [
    "Is Sam a good guy?",
    "Certainly Yes!",
    "Does Sam wear glasses?",
    "Yes"
]
trainer.train(train_list)

List Trainer: [####################] 100%


In [18]:
# Let's see what happens this time
response = chatbot.get_response("Is Sam a good guy?")
print(response)

Certainly Yes!


# 4. Solve a Math Problem by Chatterbot

# 4.1. Add the math logic_adapters into the attribute 'logic_adapters'

In [19]:
chatbot = ChatBot('chatbot',
                  logic_adapters=[
                      'chatterbot.logic.BestMatch',
                      'chatterbot.logic.MathematicalEvaluation'
                  ])

# 4.2. Ask a math question

In [20]:
# There are several logic_adapters in the chatterbot.logic package. Try explore for more.
response = chatbot.get_response("1 * 1 + 2 - 3 / 5?")
print(response)

1 * 1 + 2 - 3 / 5 = 2.4


# 5. Train an ASD FAQ Bot based on Q&A conversation pairs in ASD FAQ KB v001.xlsx

# 5.1. Create a ListTrainer for training

In [21]:
trainer = ListTrainer(chatbot)

# 5.2. Load dataset

In [22]:
data = pd.read_excel('./ASD FAQ KB v001.xlsx', sheet_name='FAQ', engine='openpyxl')
question = data.get('Question')
answer = data.get('Long_Answer')
print("Number of Q&A Pairs is :", len(data))

Number of Q&A Pairs is : 226


# 5.3. Iteratively learning/storing the question and answer (into Sqlite DB)

### Prepare training Q&A pairs as python list:

In [28]:
train_list = []
for i in range(len(question)):
    train_list.append(question[i])
    train_list.append(answer[i])

train_list[0:6] # Display first few Q&A in list form:

['What are Autism Spectrum Disorders (ASD)?',
 'ASD refers to a wide spectrum of neurodevelopmental disorders that affect communication, behavior, and social interaction. Children with ASD have delayed or absent communication skills and may demonstrate repetitive or idiosyncratic use of language. They often have underdeveloped play skills and may engage in repetitive behaviors or adhere to rigid routines. For more information about the warning signs for autism, visit the U.S. Centers for Disease Control and Prevention website at www.cdc.gov/actearly.',
 'How common is autism?',
 'According to a 2020 report commissioned by the U.S. Centers for Disease Control and Prevention, approximately one in 54 American children has been diagnosed with autism or a closely related neurodevelopmental disorder—a dramatic increase over the past decade.',
 'What causes autism? Can it be cured?',
 'The causes of this complex disorder remain uncertain. Instructional methods such as Applied Behavior Analysi

### Training/Learning:

In [30]:
trainer.train(train_list)

List Trainer: [####################] 100%


### Now user can interact with our trained/learnt ASD FAQ Bot

In [22]:
response = chatbot.get_response("What is autism spectrum disorder?")
print(response)

Autism spectrum disorder (ASD) refers to a group of complex neurodevelopment disorders characterized by repetitive and characteristic patterns of behavior and difficulties with social communication and interaction. The symptoms are present from early childhood and affect daily functioning. The term “spectrum” refers to the wide range of symptoms, skills, and levels of disability in functioning that can occur in people with ASD. Some children and adults with ASD are fully able to perform all activities of daily living while others require substantial support to perform basic activities. The Diagnostic and Statistical Manual of Mental Disorders (DSM-5, published in 2013) includes Asperger syndrome, childhood disintegrative disorder, and pervasive developmental disorders not otherwise specified (PDD-NOS) as part of ASD rather than as separate disorders. A diagnosis of ASD includes an assessment of intellectual disability and language impairment. ASD occurs in every racial and ethnic group

In [23]:
response = chatbot.get_response("How common is autism spectrum disorder?")
print(response)

The Center for Disease Control and Prevention’s (CDC) Autism and Developmental Disabilities Monitoring (ADDM) Network released data in 2007 that found about 1 in 150 8-year-old children in the areas of the United States they monitor had an ASD1 For more on the numbers: Prevalence of ASDs, from the CDC. Autism occurs throughout the world in families of all racial, ethnic and social backgrounds. Boys are four times more likely to have autism than girls2. The MIND Institute in California found that the number of cases of autism is on the rise. This is probably a national trend.


# 6 Workshop

# 6.1. Supervised learning chatterbot

In [11]:
# Build a supervised learning chatterbot
# Requirement: Every time the FAQ bot responses, the user need to provide feedback on 
# whether the answer is good or not.
#     If good, FAQ bot learns (stores) the response.
#     If not good, user inputs a new answer for the learning bot to learn.

learning_bot = ChatBot('Supervised Learning Bot New')
trainer = ChatterBotCorpusTrainer(learning_bot)
trainer.train("chatterbot.corpus.english")

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [14]:
# =========code start==========
key = True
trainer = ListTrainer(learning_bot)
while(key):
    user = input('User: ')
    response = chatbot.get_response(user)
    print("Bot:",response)
    feedback = input('Bot: Is my response satisfactory?(y/n): ')
    
    if(feedback=='n'):
        print("Bot: Sorry for the inconvenience !!")
        print("Bot: What should have been my response?")
        user_response = input("User: ")
        train_list=[user,user_response]
        trainer.train(train_list)
        
    check = input("Bot: Do you have any more questions?(y/n): ")
    
    if(check=='n'):
        key=False
# =========code end============

User: Hi
Bot: Hello
Bot: Is my response satisfactory?(y/n): y
Bot: Do you have any more questions?(y/n): y
User: Is Captain America stronger or Hulk?
Bot: I work on all kinds of computers, Mac, IBM or UNIX. it doesn't matter to me.
Bot: Is my response satisfactory?(y/n): n
Bot: Sorry for the inconvenience !!
Bot: What should have been my response?
User: Hulk
List Trainer: [####################] 100%
Bot: Do you have any more questions?(y/n): y
User: Is Captain America stronger or Hulk?
Bot: Hulk
Bot: Is my response satisfactory?(y/n): y
Bot: Do you have any more questions?(y/n): n
